In [1]:
!pip install opencv-python
!pip install ipynb
!pip install python-math
!pip install matplotlib
!pip install numpy

In [2]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

# VISIONINPUT MODULE

In [3]:
#DUMMY VISION INPUT (does not include fov and res calib)

class VisionInput:
    frame = None
    
    def __init__(self, res, FOV): #a little confused abt the importance of FOV and resolution w/out calibration in the detection pipeline...? - Bhargav
        self.res = res
        self.FOV = FOV
        
        self.cap = cv2.VideoCapture(0)

        if not self.cap.isOpened():
            print("failed to init camera")
            exit

    def calibrate():
        pass

    def start(self):
        global frame

        while True:
            cv2.imshow('frame', self.getFrame())
            if (cv2.waitKey(1) == ord('q')):
                break
            
            #for testing purposes
            if (cv2.waitKey(1) == ord('p')):
                print(self.getFrame())
            
        self.cap.release()
        cv2.destroyAllWindows()
    

    def getFrame(self):
        ret, frame = self.cap.read()
        
        
        if not ret:
            print('frame not reached')
            exit
        
        #frame = cv2.resize(frame, (600, int(600*frame.shape[1]/frame.shape[0])))

        return np.asarray(frame)

In [4]:
#TEST BLOCK

vi = VisionInput(0, 0)
vi.start()

# DETECTION MODULE

In [56]:
#NOT DONE

class Target:
    def __init__(self, contour, target_type):
        self.contour = contour
        self.target_type = target_type
    
    def __str__(self):
        return self.target_type + " coords - " + self.contour

    def getType(self):
        return self.target_type

    def getContour(self):
        return self.contour
    
    def computeYawAngle(R):
        pass

    def computePitchAngle(R):
        pass

In [ ]:
#NOT DONE

class Colored(Target):
    def __init__(self, color):
        self.color = color

    def getType(self):
        return "Colored"
    

In [60]:
#NOT DONE

class AprilTag(Target):
    def __init__(self, id):
        self.id = id
    
    def getType(self):
        return "AprilTag"

    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def getAngle(self):
        return self.angle
    
    def getID(self):
        return self.id

In [58]:
class Detector:

    def detect(self, array, objectsToDetect):
        color_dict_HSV = {'black': [[180, 255, 30], [0, 0, 0]],
              'white': [[180, 18, 255], [0, 0, 231]],
              'red1': [[180, 255, 255], [159, 50, 70]],
              'red2': [[9, 255, 255], [0, 50, 70]],
              'green': [[89, 255, 255], [36, 50, 70]],


              #merge of blue and purple
              'cube': [[158, 255, 255], [90, 50, 70]],
              'cone': [[30, 255, 255], [21, 50, 70]]
        }
        #================================================================================

        frame = cv2.resize(array, (500, int(500*array.shape[0]/array.shape[1])))
        results = dict(zip(objectsToDetect, [None for i in range(len(objectsToDetect))]))

        #CUBE DETECTION

        colors = {
            "CUBE": color_dict_HSV['cube'],
            "CONE": color_dict_HSV['cone']
        }
        
        for object in objectsToDetect:
            hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv_frame, np.array(colors[object][1]), np.array(colors[object][0]))

            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
            morph = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
            contours, hier = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            x = 0
            y = 0
            w = 0
            h = 0

            for contour in contours:
                tx,ty,tw,th = cv2.boundingRect(contour)
                print(tx, ty, tw, th)
                if (tw * th > w * h and not (tx == 0 and ty == 0 and tw == frame.shape[1] and th == frame.shape[0])):
                    x = tx
                    y = ty
                    w = tw
                    h = th

            if (object in objectsToDetect): results[object] = [x, y, w, h]

            print("X: %2d, Y: %2d, W: %2d, H: %2d" % (x, y, w, h))

            #annotate contour
            cv2.rectangle(frame, (x,y), (x + w, y + h), (0, 0, 255), 2)
            cv2.circle(frame, (int(x + w/2), int(y + h/2)), radius = 0, color = (0, 0, 255), thickness=5)
            cv2.putText(frame, object, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2, cv2.LINE_AA)

            while True:
                cv2.imshow("o", frame)
                if cv2.waitKey(0):
                    break
            
            cv2.destroyAllWindows()

            results[object] = Target([x, y, w, h], object)

        return results

In [59]:
img = cv2.imread("test-imgs/cone.png")

d = Detector()
d.detect(np.asarray(img), ["CUBE", "CONE"])

X:  0, Y:  0, W:  0, H:  0
159 600 1 1
451 587 3 1
474 582 26 14
463 582 1 1
439 582 12 1
137 571 1 1
448 569 9 1
443 564 1 1
169 562 1 1
458 559 9 6
279 551 8 5
440 550 7 2
373 547 1 1
375 545 1 1
450 544 5 4
353 544 1 1
357 543 1 1
295 543 13 8
109 542 2 1
318 539 1 2
441 535 1 1
438 532 1 1
374 524 5 1
498 499 2 1
487 482 1 1
454 475 2 4
488 458 1 1
483 453 1 1
476 447 1 1
447 442 1 1
440 436 1 1
414 419 2 2
450 413 1 1
413 412 1 1
428 402 1 1
415 376 1 3
429 313 1 3
469 309 1 1
440 299 1 1
421 293 1 1
99 106 307 425
X: 99, Y: 106, W: 307, H: 425
{'CUBE': <__main__.Target object at 0x000001970BE836D0>, 'CONE': <__main__.Target object at 0x000001970B31EC90>}
